In [1]:
from itkwidgets import view
import numpy as np
import sys
sys.path.append("/users/jfavre/Projects/ParaView/Python")
from vtkGridConstructors import *

# Tested Thu May 13 19:10:18 CEST 2021

# Create a Spherical Grid

In [2]:
dims = [5, 9, 7] # radius, theta(east-west longitude), phi(north-south latitude)

Raxis = np.linspace(1., 2., dims[0])

# only use 3/4 of the full longitude in order to view the inside of the sphere
Thetaaxis = np.linspace(0.,np.pi*1.5, dims[1])

Phiaxis = np.linspace(0.,np.pi*1.0, dims[2])

p, t, r = np.meshgrid(Phiaxis, Thetaaxis, Raxis, indexing="ij")
X = r * np.cos(t) * np.sin(p)
Y = r * np.sin(t) * np.sin(p)
Z = r * np.cos(p)

G = vtkStructuredGridFromArrays()

G.SetCoordinates(X, Y, Z)
G.AddArray(r.ravel(), "radius")
G.AddArray(t.ravel(), "theta")
G.AddArray(p.ravel(), "phi")
# reduce dimension of array by 1 in all directions
G.AddCellArray(p[1:,1:,1:].ravel(), "phi")

V = algs.make_vector(X.ravel(),
                   Y.ravel(),
                   Z.ravel())
                   
# V is a 2D array of shape(np.prod(global_dims), 3)
G.AddArray(V, "Velocity")

# reduce dimension of array by 1 in all directions, except the last to keep velocity a 3-tuple
Vcell = V.reshape(dims[2],dims[1],dims[0],3)[1:,1:,1:,:].reshape((dims[0]-1)*(dims[1]-1)*(dims[2]-1), 3)
G.AddCellArray(Vcell, "Velocity")

In [3]:
# generate an outline bounding box for the mesh
geom = vtk.vtkGeometryFilter()
geom.SetInputData(G.GetOutput())
                        
features = vtk.vtkFeatureEdges()
features.SetInputConnection(geom.GetOutputPort())
features.BoundaryEdgesOn()
features.FeatureEdgesOn()
features.Update()
edges = features.GetOutput()

In [4]:
v0 = view(geometries=[G.GetOutput(), edges])
v0

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# A cartesian grid, a.k.a. vtkImageData

In [5]:
dims = [64,64,64]
bounds = (-10.0, 10.0, -10.0, 10.0,-10.0, 10.0)
xaxis = np.linspace(bounds[0], bounds[1], dims[0])
yaxis = np.linspace(bounds[2], bounds[3], dims[1])
zaxis = np.linspace(bounds[4], bounds[5], dims[2])
[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = xc**2 + yc**2 + zc**2

from vtk.util.vtkImageImportFromArray import vtkImageImportFromArray
iD = vtkImageImportFromArray()
iD.SetArray(data) # it is given the name "scalars" by default
iD.SetDataOrigin((bounds[0], bounds[2], bounds[4]))
iD.SetDataSpacing(((bounds[1]-bounds[0])/(dims[0]-1),
                   (bounds[3]-bounds[2])/(dims[0]-1),
                   (bounds[5]-bounds[4])/(dims[0]-1)))
iD.Update()
iD.GetOutput().GetPointData().SetActiveScalars("scalars")

print("mesh dimensions = ", iD.GetOutput().GetDimensions())
assert iD.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = iD.GetOutput().GetPointData().GetArray(0).GetRange()
print("data scalar range = ", datarange)

# generate an outline bounding box for the mesh
geom = vtk.vtkGeometryFilter()
geom.SetInputData(iD.GetOutput())
                        
features = vtk.vtkFeatureEdges()
features.SetInputConnection(geom.GetOutputPort())
features.BoundaryEdgesOn()
features.FeatureEdgesOn()
features.Update()
edges = features.GetOutput()

mesh dimensions =  (64, 64, 64)
data scalar range =  (0.07558578987150362, 300.0)


In [6]:
#view(geometries=[iD.GetOutput(), edges])
v1 = view(image=iD.GetOutput(), geometries=[edges])
v1

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Create a cylindrical - structured grid

In [ ]:
dims = [13,27,15] # Radius dimension, Theta dimension, Z dimension
Raxis = np.linspace(1., 2., dims[0])
Thetaaxis = np.linspace(0.,np.pi*1.5, dims[1])
Zaxis = np.linspace(0., 2.0, dims[2])
Z, t, r = np.meshgrid(Zaxis, Thetaaxis, Raxis, indexing="ij")
X = r * np.cos(t)
Y = r * np.sin(t)

NsG = vtkStructuredGridFromArrays()
NsG.SetCoordinates(X, Y, Z)
NsG.AddArray(r.ravel(), "radius")
NsG.AddArray(Z.ravel(), "z-elevation")
NsG.AddArray(t.ravel(), "angle")

print("StructuredGrid mesh dimensions = ", NsG.GetDimensions())
print("StructuredGrid mesh bounds = ", NsG.GetOutput().GetBounds())
assert NsG.GetOutput().GetPointData().GetArray(1).GetName() == 'z-elevation'
datarange = NsG.GetOutput().GetPointData().GetArray("z-elevation").GetRange()
print("StructuredGrid data scalar range = ", datarange)

# generate an outline bounding box for the mesh
geom = vtk.vtkGeometryFilter()
geom.SetInputData(NsG.GetOutput())
                        
features = vtk.vtkFeatureEdges()
features.SetInputConnection(geom.GetOutputPort())
features.BoundaryEdgesOn()
features.FeatureEdgesOn()
features.Update()
edges = features.GetOutput()

In [ ]:
v2 = view(geometries=[NsG.GetOutput(), edges])
v2

# Create a point cloud

In [ ]:
nbpts = 10000
coords = np.random.random_sample((nbpts, 3))

PC = vtkPointSetFromArrays()
#PC.SetCoordinates(coords[:,0], coords[:,1], coords[:,2])
PC.SetVCoordinates(coords)
PC.AddArray(coords[:,0]*coords[:,0] + coords[:,1]*coords[:,1] + coords[:,2]*coords[:,2], "distancesq")
PC.AddArray(coords[:,0], "x-coords")

print("Point Cloud has", PC.GetOutput().GetNumberOfPoints(), "vertices")


In [ ]:
v3 = view(point_sets=[PC.GetOutput()])
v3

# Create a Rectilinear Grid

In [ ]:
dims = [64,64,64] # Radius dimension, Theta dimension, Z dimension
xaxis = np.linspace(-.5, 1., dims[0])
#xaxis = xaxis**2
yaxis = np.linspace(0., 1., dims[1])
yaxis = np.sqrt(yaxis)
zaxis = np.linspace(0., 1., dims[2])

[xc,yc,zc] = np.meshgrid(zaxis,yaxis,xaxis, indexing="ij")
data = np.sqrt(xc**2 + yc**2 + zc**2)

rG = vtkRectGridFromArrays()
rG.SetCoordinates(xaxis, yaxis, zaxis)
rG.AddArray(data.ravel(), "scalars")

print("RectilinearGrid dimensions = ", rG.GetDimensions())
assert rG.GetOutput().GetPointData().GetArray(0).GetName() == 'scalars'
datarange = rG.GetOutput().GetPointData().GetArray("scalars").GetRange()
print("RectilinearGrid  data scalar range = ", datarange)

# generate an outline bounding box for the mesh
geom = vtk.vtkGeometryFilter()
geom.SetInputData(rG.GetOutput())
                        
features = vtk.vtkFeatureEdges()
features.SetInputConnection(geom.GetOutputPort())
features.BoundaryEdgesOn()
features.FeatureEdgesOn()
features.Update()
edges = features.GetOutput()

In [ ]:
v4 = view(geometries=[rG.GetOutput(), edges])
v4